In [1]:
using StaticArrays, DifferentialEquations, DynamicalSystems, CairoMakie, JLD

In [2]:
function sigma(x)
    return 1.0 / ( 1.0 + exp( -10.0 * ( x  - ( - 0.25 ) ) ) )
end

function HR(u, p, t)
        
    a, b, c, d, s, xr, r,  I, vs, chem_link, el_link  = p
    x1, y1, z1, x2, y2, z2 = u
    
    du1 = y1 + b * x1 ^ 2 - a * x1 ^3 - z1 + I - chem_link * ( x1 - vs ) * sigma(x2) + el_link * ( x2 - x1 )
    du2 = c - d * x1 ^2 - y1
    du3 = r * ( s * ( x1 - xr ) - z1 )
    
    du4 = y2 + b * x2 ^ 2 - a * x2 ^3 - z2 + I - chem_link * ( x2 - vs ) * sigma(x1) + el_link * ( x1 - x2 )
    du5 = c - d * x2 ^2 - y2
    du6 = r * ( s * ( x2 - xr ) - z2 )
    return SVector(du1, du2, du3,
                    du4, du5, du6)
end

HR (generic function with 1 method)

In [3]:
function output(idx, parameter, u0)
    println("index: $idx; chemical_link: $parameter");flush(stdout)
    println("u0: $u0");flush(stdout)
end
function output(Λ, u0)
    println("Λ: $Λ");flush(stdout)
    println("last point: $u0");flush(stdout)
end
function separate()
    println("----------------");flush(stdout)
    println("  ");flush(stdout)
end

separate (generic function with 1 method)

In [4]:
a = 1.; b = 3.; c = 1.; d = 5.;
xr = -1.6; r = 0.01; s = 5.; I = 4.; xv = 2.;
electrical_link = 0.01

0.01

In [5]:
tf = 20000.0
tt = 20000.0
tstep = 0.01
integ_setting = (alg = Vern9(), adaptive = false, dt = 0.01);

In [9]:
# chemical link
start_p = -0.02
finish_p = 0.025
p_range = range(start_p, finish_p, length = 500);

In [10]:
Λs =  zeros(length(p_range), 6)
u0s = zeros(length(p_range), 6);

In [ ]:
for (idx, chemical) in enumerate(p_range)

    if idx == 1
        global u0 = SA[-2.5, 0.0, 0.0, -1.5, 0.0, 0.0]
    end

    output(idx, chemical, u0)

    p = SA[a, b, c, d, s, xr, r, I, xv, chemical, electrical_link]
    ds = ContinuousDynamicalSystem(HR,  u0, p)

    tr = trajectory(ds, 500.0, Δt = tstep; Ttr = tt, diffeq = integ_setting)
    u0 = tr[end]; ds = ContinuousDynamicalSystem(HR,  u0, p)
    Λ = lyapunovspectrum(ds, tf; diffeq = integ_setting)
    Λs[idx, :] = Λ
    u0s[idx, :] = u0
    
    output(Λs[idx, :], u0s[idx, :])
    separate()
    
end

index: 1; chemical_link: -0.02
u0: [-2.5, 0.0, 0.0, -1.5, 0.0, 0.0]
Λ: [0.00564775981724202, 0.0002593068211352955, -0.005742795740550929, -0.013339103605778248, -7.752964091574449, -9.74094835676635]
last point: [-0.8682730036388684, -2.9130955582156, 3.9840914099164544, -0.8429839761569174, -2.708639475566594, 4.000513711691686]
----------------
  
index: 2; chemical_link: -0.019909819639278556
u0: [-0.8682730036388684, -2.9130955582156, 3.9840914099164544, -0.8429839761569174, -2.708639475566594, 4.000513711691686]
Λ: [0.004047155679038945, 2.3331302074266622e-5, -0.004759906526053787, -0.012659676476230874, -7.745994766811147, -9.76396885292307]
last point: [-1.0650913326548637, -4.488005568107949, 4.142014144942948, -0.9427995234294235, -3.2790643354701086, 4.242962449990328]
----------------
  
index: 3; chemical_link: -0.019819639278557115
u0: [-1.0650913326548637, -4.488005568107949, 4.142014144942948, -0.9427995234294235, -3.2790643354701086, 4.242962449990328]
Λ: [0.004788008

  
index: 19; chemical_link: -0.018376753507014027
u0: [-1.2006879639410952, -6.344289162763394, 3.699239980049475, -1.1286013011681875, -5.5756695303416, 3.661863707798871]
Λ: [0.004515095255402298, 0.00029832546424646186, -0.004652088738484883, -0.013603460265000974, -7.851836420509312, -9.65715723823616]
last point: [-0.29539980566476914, -0.02345805274763376, 4.045882980693178, -0.7609899135751258, -2.1268109449648906, 4.003613966836844]
----------------
  
index: 20; chemical_link: -0.018286573146292586
u0: [-0.29539980566476914, -0.02345805274763376, 4.045882980693178, -0.7609899135751258, -2.1268109449648906, 4.003613966836844]
Λ: [0.004114615894969054, -0.00012023656579054154, -0.004942609278411632, -0.01323553569596663, -7.833350140791575, -9.664353011204932]
last point: [-0.8629150935843666, -3.218529715034153, 3.592052494222207, -0.9555681805528736, -3.7708658966404536, 3.860324867630523]
----------------
  
index: 21; chemical_link: -0.01819639278557114
u0: [-0.862915093584

----------------
  
index: 37; chemical_link: -0.016753507014028057
u0: [-0.9895873251138123, -3.7170636914668567, 4.209422921346541, -0.9149346697710712, -3.0453100201609646, 4.247806434376814]
Λ: [0.0038038959615771884, 0.00022431562219437493, -0.006149626286190584, -0.014399072821889583, -7.97778169945094, -9.52458469037015]
last point: [-1.0345555760899883, -4.169491929127559, 4.1666516108063325, -1.030772651812783, -4.113966078987257, 4.188389972795516]
----------------
  
index: 38; chemical_link: -0.016663326653306613
u0: [-1.0345555760899883, -4.169491929127559, 4.1666516108063325, -1.030772651812783, -4.113966078987257, 4.188389972795516]
Λ: [0.0027444390719664337, 2.8600759378306476e-5, -0.003403578355843307, -0.015231067368089416, -8.008893727731975, -9.500197055758521]
last point: [-1.1941490186504438, -6.004436036517246, 3.9855956438441367, -1.2103855173476177, -6.181169100452734, 3.998054613765388]
----------------
  
index: 39; chemical_link: -0.016573146292585172
u0: [-

last point: [0.054099134135192836, 0.4906988772735791, 3.9154911972323143, 0.6217580933754844, 0.4117982415073297, 3.94986617295933]
----------------
  
index: 55; chemical_link: -0.015130260521042084
u0: [0.054099134135192836, 0.4906988772735791, 3.9154911972323143, 0.6217580933754844, 0.4117982415073297, 3.94986617295933]
Λ: [0.002743995640636083, 6.61604916470087e-5, -0.0008642474249755675, -0.01996205781941215, -8.658756323392446, -8.859941356176824]
last point: [-1.166073033998215, -5.62327985518619, 4.055778269592861, -1.1548299329180316, -5.477551407259603, 4.0791839621044605]
----------------
  
index: 56; chemical_link: -0.015040080160320641
u0: [-1.166073033998215, -5.62327985518619, 4.055778269592861, -1.1548299329180316, -5.477551407259603, 4.0791839621044605]
Λ: [0.0024613773713385613, 0.0001334149628563649, -0.000840255637426751, -0.01958860833581261, -8.654263306129371, -8.857136172143214]
last point: [-0.9433825904500935, -3.4457988509485924, 4.063322429226274, -0.94161

Λ: [0.002794447935901374, 0.00077171577847968, -0.0004957703064937981, -0.018873554303677424, -8.633562217163636, -8.869476715905291]
last point: [-0.8042064061676255, -2.7841207059569184, 3.5982472095802196, -0.7934506542099539, -2.7042063445046423, 3.6037781504307884]
----------------
  
index: 73; chemical_link: -0.013507014028056112
u0: [-0.8042064061676255, -2.7841207059569184, 3.5982472095802196, -0.7934506542099539, -2.7042063445046423, 3.6037781504307884]
Λ: [0.002556134606318123, 0.0006680189285462005, -0.00014886985580599122, -0.018933471362353366, -8.642241047908431, -8.875163716014239]
last point: [-1.1408230217712798, -5.312150206635189, 4.093609298348426, -1.140737025943482, -5.314361235675736, 4.090181130378628]
----------------
  
index: 74; chemical_link: -0.01341683366733467
u0: [-1.1408230217712798, -5.312150206635189, 4.093609298348426, -1.140737025943482, -5.314361235675736, 4.090181130378628]
Λ: [0.002881347647745489, 0.0013006833131625098, -0.00040862931632966784

u0: [-0.7933491937095691, -2.5082898551197617, 3.826701272312279, -0.8070174372708911, -2.6100790985946336, 3.8190925504595308]
Λ: [0.0027850529760818318, 0.0010654847997652733, -9.616054211305046e-5, -0.020289882982646314, -8.622549360537148, -8.88058362847951]
last point: [-1.192275473257975, -6.240001604986642, 3.7065624079312767, -1.1878457170348231, -6.1879644586619005, 3.7079539359145324]
----------------
  
index: 91; chemical_link: -0.011883767535070141
u0: [-1.192275473257975, -6.240001604986642, 3.7065624079312767, -1.1878457170348231, -6.1879644586619005, 3.7079539359145324]
Λ: [0.0031951536331206224, 0.0013637398667887613, -5.990310050331074e-5, -0.02113468929263897, -8.637091169131251, -8.883273449014995]
last point: [-0.9466696599253701, -3.3216835214612113, 4.232988463354416, -0.950537422190623, -3.3546356138332567, 4.232894121019071]
----------------
  
index: 92; chemical_link: -0.011793587174348698
u0: [-0.9466696599253701, -3.3216835214612113, 4.232988463354416, -0.9

----------------
  
index: 108; chemical_link: -0.01035070140280561
u0: [-1.2290107099302152, -6.44561437367242, 3.9495046147952793, -1.2278633381929602, -6.452613194316466, 3.927093506850207]
Λ: [0.0032279948358689115, 0.0017555599704625268, -1.4567427528123055e-5, -0.018284104665260367, -8.601628593937438, -8.916010788902717]
last point: [-0.9507990173548455, -3.3637021990878595, 4.225302205120202, -0.9424150060407256, -3.2866309179608577, 4.23201515732033]
----------------
  
index: 109; chemical_link: -0.010260521042084168
u0: [-0.9507990173548455, -3.3637021990878595, 4.225302205120202, -0.9424150060407256, -3.2866309179608577, 4.23201515732033]
Λ: [0.0031489720526266542, 0.001322655351253908, -5.074094298166772e-5, -0.018262580858809913, -8.608529892839515, -8.90830621501742]
last point: [-0.9124119396520903, -3.2299829991038287, 4.018055773911011, -0.9144774696778101, -3.2502395354074385, 4.014178301734058]
----------------
  
index: 110; chemical_link: -0.010170340681362725
u0:

last point: [1.5748340027454588, -3.4803075263850847, 4.191761089896462, 0.5847236757952962, 0.4937706777515613, 4.117976469483674]
----------------
  
index: 126; chemical_link: -0.008727454909819639
u0: [1.5748340027454588, -3.4803075263850847, 4.191761089896462, 0.5847236757952962, 0.4937706777515613, 4.117976469483674]
Λ: [0.0028192789465327417, 0.00012003719929037158, -0.001105798903527229, -0.014817874065824937, -8.567923246659735, -8.952593461226936]
last point: [-1.2303744229193734, -6.607885264843798, 3.791480319333118, -1.2221531400054668, -6.509178872055472, 3.7922577658244583]
----------------
  
index: 127; chemical_link: -0.008637274549098196
u0: [-1.2303744229193734, -6.607885264843798, 3.791480319333118, -1.2221531400054668, -6.509178872055472, 3.7922577658244583]
Λ: [0.002212369079438643, 0.0004130469607089145, -0.0011378681043850846, -0.015107792817024847, -8.568241921757695, -8.946899985713687]
last point: [-0.9254097676819757, -3.501049774258939, 3.8335741002737613,

Λ: [0.0017181727895426199, 0.0003496701013361218, -0.000744236672129435, -0.015872444830970438, -8.543356338795109, -8.975261774740373]
last point: [-1.2294810970303136, -6.461466660162119, 3.9384518276460496, -1.224710365803321, -6.382095447355105, 3.962839899986487]
----------------
  
index: 144; chemical_link: -0.007104208416833667
u0: [-1.2294810970303136, -6.461466660162119, 3.9384518276460496, -1.224710365803321, -6.382095447355105, 3.962839899986487]
Λ: [0.0018983132362879469, 5.962973728715599e-5, -0.001011284913001443, -0.01599329871648438, -8.547504545552854, -8.980110409563936]
last point: [-0.934492547206487, -3.2119909702506093, 4.241598902856942, -0.9201586669055847, -3.0894610075217814, 4.24636395338099]
----------------
  
index: 145; chemical_link: -0.0070140280561122245
u0: [-0.934492547206487, -3.2119909702506093, 4.241598902856942, -0.9201586669055847, -3.0894610075217814, 4.24636395338099]
Λ: [0.001700229290363459, 0.0003683499657726795, -0.00022035761774553, -0.0

In [ ]:
f = Figure(resolution = (1600, 400))
axis = Axis(f[1, 1])
lines!(axis, p_range, Λs[:, 1], color = :red)
lines!(axis, p_range, Λs[:, 2], color = :orange)
lines!(axis, p_range, Λs[:, 3], color = :blue)
f

In [ ]:
save("LSE_electrical.jld", "data",Λs)
save("u0_electrical.jld", "data", u0s)